This notebook is for automatically extracting a single 3d surface from multiple slice-wise 2d segmentations.
It relies on the Hausdorff distance, and a single given seed annotation.

It works by iterating over every slice (marching away from the seeded slice), finding the annotation with the closest hausdorff distance to the current one, then repeating this process until the end of the cell is reached.

In [3]:
import data_loader
from distance_measures import hausdorff_vec
import numpy as np
import matplotlib.pyplot as plt
from os.path import join, exists
from os import makedirs

In [4]:
data = data_loader.DataLoader()  # will take a while to run as big df is read

In [5]:
roi_id = 'ROI_1656-6756-329'
img_volume = data_loader.get_img_volume(roi_id)

##### Extract similar lines given initial seed

In [7]:
num_slices, width, height = img_volume.shape

chosen_seed_slice = 150     # axial/z index of seed slice
chosen_seed_annotation = 8  # which annotation is the best for the seed slice?

# ??
seed_slice = 150     # axial/z index of seed slice
seed_annotation = 8  # which annotation is the best for the seed slice?

annotation_set = {s: None for s in range(num_slices)}
annotation_set[seed_slice] = seed_annotation  # list of predicted best annotations per slice

In [8]:
def process(range_):
  
  seed_slice = chosen_seed_slice
  seed_annotation = chosen_seed_annotation
  slice_annotations = data.get_annotations(roi_id, seed_slice)
  best_annotation_vec = data_loader.annotation_to_vec(slice_annotations[seed_annotation])

  for seed_slice in range_:
    min_hausdorff = np.inf  # smallest found hausdorff distance between previous slice and this one
    min_hausdorff_annotation_index = 0  # index of the annotation in this slice that is closest to previous
    slice_annotations = data.get_annotations(roi_id, seed_slice)
    if not slice_annotations:
      break

    for annotation_index, annotation in enumerate(slice_annotations):
      candidate_vec = data_loader.annotation_to_vec(annotation)

      if len(candidate_vec) == 0: # empty vector
        continue
      elif len(candidate_vec) > 20000: # too many, will cause memory overflow.
        step = np.ceil(len(candidate_vec) / 20000)
        candidate_vec = candidate_vec[::int(step)] # sample...

      haudorff_distance = hausdorff_vec(best_annotation_vec, candidate_vec)    
      if haudorff_distance < min_hausdorff:
        min_hausdorff = haudorff_distance
        min_hausdorff_annotation_index = annotation_index

    annotation_set[seed_slice] = min_hausdorff_annotation_index
    best_annotation_vec =  data_loader.annotation_to_vec(slice_annotations[min_hausdorff_annotation_index])
#   print(f'closest to {seed_slice} is {min_hausdorff_annotation_index} with distance {min_hausdorff}')

In [9]:
process(range(chosen_seed_slice+1, num_slices))
process(reversed(range(0, chosen_seed_slice)))

##### Compile discovered annotation set into image volume
I realise that process() could call a func to save annotation images, which would be more efficient.

In [ ]:
dpi=200
x_scale = width / 1000
y_scale = height / 1000
fig = plt.figure(frameon=False, figsize=(width//dpi, height//dpi), dpi=dpi)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

def save_annotations_as_images(save_dir, annotation_set):
  
  for slice_num in range(num_slices):
  
    annotation_index = annotation_set[slice_num]
    
    if annotation_index is not None:
      ax.clear()
      canvas = np.zeros((width, height))
      ax.imshow(canvas, cmap='gray')
      current_slice_annotations = data.get_annotations(roi_id, slice_num)
      list_of_vectors = [np.array(eval(points)) for points in current_slice_annotations[annotation_index]['points']]
      for vec in list_of_vectors:
        vec[:, 0] *= x_scale
        vec[:, 1] *= y_scale
        ax.plot(vec[:, 0], vec[:, 1], color='white', alpha=1, linewidth=1.0, antialiased=False)

    save_path = join(save_dir, str(slice_num)+'.png') 
    fig.savefig(save_path)

In [ ]:
ROOT_SAVE_DIR = 'hausdorff_marching_aggregations'
save_dir = join(ROOT_SAVE_DIR, roi_id)
if not exists(save_dir):
  makedirs(save_dir)  
save_annotations_as_images(save_dir, annotation_set)